# Ähnlichkeitsmaße für Gesten

In [ ]:
import os
import sys
import numpy as np
import scipy.spatial.distance as dis
import scipy.stats as st
import time

from glob import glob
from gesture_comparison_helper import load_gestures
from itertools import combinations, combinations_with_replacement

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Utils.data_preparation_helper import prepare_data

In [ ]:
def calculate_similarity(a, b, sim_type):
    if sim_type == 'cosine':
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif sim_type == 'euclid':
        return 1/(1+dis.euclidean(a, b))
    elif sim_type == 'pearson':
        return st.pearsonr(a, b)[0]
    else:
        print('sim_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(sim_type))

In [ ]:
FRAME_NUMBER = 3
sensors = ['accelerometer', 'gyro', 'emg', 'orientation']
sensor_weights = [0.25, 0.25, 0.25, 0.25]

own = ['big_stop', 'come_down', 'come_here', 'come_up', 'flip', 'small_stop']
paper = ['hcwh', 'hcwv', 'hcws',
         'hcch', 'hccv', 'hccs',
         'tcwh', 'tcwv', 'tcws',
         'tcch', 'tccv', 'tccs',
         'lcwh', 'lcwv', 'lcws',
         'lcch', 'lccv', 'lccs']

## Vergleiche gleiche Gesten

In [ ]:
print('Mean values of different similarity measurements\n')

for gesture in own + paper:
    gesture_dicts = load_gestures(glob('../Data/converted/*/' + gesture + '/*'))
    
    print('Gesture: {}'.format(gesture))
    print('-'*20)
    for sensortype in sensors:
        feature_vectors = prepare_data(gesture_dicts, FRAME_NUMBER, sensortype, verbose=False)[0]

        cosine_simi = []
        euclid_simi = []
        pearson_simi = []

        for gesture in combinations(feature_vectors, 2):
            cosine_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='cosine'))
            euclid_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='euclid'))
            pearson_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='pearson'))

        print('Sensor type: {}'.format(sensortype))
        print('Cosine similarity: {}'.format(np.mean(cosine_simi)))
        print('Euclid similarity: {}'.format(np.mean(euclid_simi)))
        print('Pearson similarity: {}'.format(np.mean(pearson_simi)))
        print()
    print()

In [ ]:
start_time = time.time()
gesture_dicts = [load_gestures(glob('../Data/converted/*/' + gesture + '/*')) for gesture in own+paper]

prepared_groups = []
for gesture_group in gesture_dicts:
    gesture = []
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'accelerometer', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'gyro', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'emg', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'orientation', verbose=False)[0])
    prepared_groups.append(gesture)
        
print('Duration: {}'.format(time.time() - start_time))

In [ ]:
for gesture, gesture_label in zip(combinations_with_replacement(prepared_groups, 2),
                                  combinations_with_replacement(own+paper, 2)):
    sensor_similarity_cosine = []
    sensor_similarity_euclid = []
    sensor_similarity_pearson = []
    
    print('{} to {}'.format(gesture_label[0], gesture_label[1]))
    for sensortype_ga, sensortype_gb in zip(gesture[0], gesture[1]):
        
        mean_a = np.divide(np.sum(sensortype_ga, axis=0), len(sensortype_ga))
        mean_b = np.divide(np.sum(sensortype_gb, axis=0), len(sensortype_gb))
        
        sensor_similarity_cosine.append(calculate_similarity(mean_a, mean_b, sim_type='cosine'))
        sensor_similarity_euclid.append(calculate_similarity(mean_a, mean_b, sim_type='euclid'))
        sensor_similarity_pearson.append(calculate_similarity(mean_a, mean_b, sim_type='pearson'))
        

    result_cosine = np.sum(np.multiply(sensor_similarity_cosine, sensor_weights))
    result_euclid = np.sum(np.multiply(sensor_similarity_euclid, sensor_weights))
    result_pearson = np.sum(np.multiply(sensor_similarity_pearson, sensor_weights))
    
    print('Weighted Cosine similarity: {}'.format(result_cosine))
    print('Weighted Euclid similarity: {}'.format(result_euclid))
    print('Weighted Pearson similarity: {}'.format(result_pearson))
    print()